### Word Group In Youtube

In [229]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [230]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [231]:
# language pair
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 40000  # native word end index
unique_word_ratio = 90  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
include_wordgroup_ratio = 90  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
word_usage_min = 5  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 45  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [232]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [233]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [234]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [235]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [236]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [237]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [238]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
317922,puisetil,5
317923,shouzheng,5
317924,steinam,5
317925,cochezle,5


In [239]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [240]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
39995,mappellera,342
39996,jinterroge,342
39997,matait,342
39998,titille,342


In [241]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [242]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [243]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

39987

#### Youtube Sentence Analysis

In [244]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:00.200,perrosguirec bretagne,BU7DYGMSUA4
1,00:00:30.972,00:00:33.609,located in the regional natural park of armorica,4nmWfnmVgo4
2,00:00:34.263,00:00:36.866,the huelgoat forest,4nmWfnmVgo4
3,00:00:37.315,00:00:39.315,has always inspired popular stories,4nmWfnmVgo4
4,00:00:40.132,00:00:43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4
...,...,...,...,...
385309,00:03:54.600,00:03:56.240,ou bien encore des échanges de pratiques,oqXlP80c1uk
385310,00:03:56.240,00:03:58.800,si vous souhaitez plus dinformations sur ces t...,oqXlP80c1uk
385311,00:03:59.200,00:04:01.260,participez à nos ateliers dinformation,oqXlP80c1uk
385312,00:04:01.900,00:04:03.260,consultez nos fascicules,oqXlP80c1uk


In [245]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [246]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4
...,...,...,...,...,...
385309,385309,234.600,236.240,ou bien encore des échanges de pratiques,oqXlP80c1uk
385310,385310,236.240,238.800,si vous souhaitez plus dinformations sur ces t...,oqXlP80c1uk
385311,385311,239.200,241.260,participez à nos ateliers dinformation,oqXlP80c1uk
385312,385312,241.900,243.260,consultez nos fascicules,oqXlP80c1uk


In [247]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [248]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [249]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4,50.0,perrosguirec,bretagne
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4,50.0,"natural, regional, armorica, located","in, of, the, park"
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4,66.6,huelgoat,"the, forest"
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4,40.0,"stories, inspired, popular","has, always"
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4,66.7,"names, refer, mysterious","to, arthur, king, with, sometimes, places"
...,...,...,...,...,...,...,...,...
385309,385309,234.600,236.240,ou bien encore des échanges de pratiques,oqXlP80c1uk,100.0,,"ou, bien, des, encore, échanges, pratiques, de"
385310,385310,236.240,238.800,si vous souhaitez plus dinformations sur ces t...,oqXlP80c1uk,87.5,thématiques,"si, plus, ces, sur, vous, souhaitez, dinformat..."
385311,385311,239.200,241.260,participez à nos ateliers dinformation,oqXlP80c1uk,80.0,à,"nos, dinformation, participez, ateliers"
385312,385312,241.900,243.260,consultez nos fascicules,oqXlP80c1uk,66.6,fascicules,"nos, consultez"


In [250]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4,50.0,perrosguirec,bretagne
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4,50.0,"natural, regional, armorica, located","in, of, the, park"
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4,66.6,huelgoat,"the, forest"
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4,40.0,"stories, inspired, popular","has, always"
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4,66.7,"names, refer, mysterious","to, arthur, king, with, sometimes, places"
...,...,...,...,...,...,...,...,...
385309,385309,234.600,236.240,ou bien encore des échanges de pratiques,oqXlP80c1uk,100.0,NaN,"ou, bien, des, encore, échanges, pratiques, de"
385310,385310,236.240,238.800,si vous souhaitez plus dinformations sur ces t...,oqXlP80c1uk,87.5,thématiques,"si, plus, ces, sur, vous, souhaitez, dinformat..."
385311,385311,239.200,241.260,participez à nos ateliers dinformation,oqXlP80c1uk,80.0,à,"nos, dinformation, participez, ateliers"
385312,385312,241.900,243.260,consultez nos fascicules,oqXlP80c1uk,66.6,fascicules,"nos, consultez"


In [251]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4,50.0,perrosguirec,bretagne
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4,50.0,"natural, regional, armorica, located","in, of, the, park"
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4,66.6,huelgoat,"the, forest"
3,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4,66.7,"names, refer, mysterious","to, arthur, king, with, sometimes, places"
4,5,44.193,46.193,sometimes to the devil or the virgin,4nmWfnmVgo4,85.7,NaN,"to, the, devil, or, sometimes, virgin"
...,...,...,...,...,...,...,...,...
382119,385309,234.600,236.240,ou bien encore des échanges de pratiques,oqXlP80c1uk,100.0,NaN,"ou, bien, des, encore, échanges, pratiques, de"
382120,385310,236.240,238.800,si vous souhaitez plus dinformations sur ces t...,oqXlP80c1uk,87.5,thématiques,"si, plus, ces, sur, vous, souhaitez, dinformat..."
382121,385311,239.200,241.260,participez à nos ateliers dinformation,oqXlP80c1uk,80.0,à,"nos, dinformation, participez, ateliers"
382122,385312,241.900,243.260,consultez nos fascicules,oqXlP80c1uk,66.6,fascicules,"nos, consultez"


In [252]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-252-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
4,5,44.193,46.193,sometimes to the devil or the virgin,4nmWfnmVgo4,85.7,NaN,"to, the, devil, or, sometimes, virgin",sometimes to the devil or the virgin
5,7,48.740,53.288,the forest of huelgoat is best known for the c...,4nmWfnmVgo4,66.7,"curiosity, huelgoat, known","of, is, rocks, beauty, for, the, best, forest,...",and beauty of its rocks
13,17,11.400,15.090,parsemant son cours long de moins de,BYd2A4JSdOo,71.4,parsemant,"cours, moins, son, long, de",son cours long de moins de
14,18,15.090,20.610,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo,78.6,"dormantes, hivernale, migratoire","est, et, une, oiseaux, eaux, éloge, des, halte...",est un éloge des eaux
15,19,20.610,26.550,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo,84.6,"herbiers, roselières","et, le, refuge, ou, deau, sont, despèces, lott...",sont le refuge despèces menacées ou en régress...
...,...,...,...,...,...,...,...,...,...
382114,385304,223.080,225.580,vous pourrez choisir des pistes dactions techn...,oqXlP80c1uk,100.0,NaN,"pourrez, choisir, vous, des, dactions, pistes,...",vous pourrez choisir des pistes dactions techn...
382117,385307,229.180,231.760,telles que les objectifs et les horaires au tr...,oqXlP80c1uk,88.9,NaN,"et, les, que, horaires, telles, au, travail, o...",telles que les objectifs et les horaires au tr...
382118,385308,231.920,234.560,mais également humaines avec des formations,oqXlP80c1uk,100.0,NaN,"également, formations, mais, des, avec, humaines",mais également humaines avec des formations
382119,385309,234.600,236.240,ou bien encore des échanges de pratiques,oqXlP80c1uk,100.0,NaN,"ou, bien, des, encore, échanges, pratiques, de",ou bien encore des échanges de pratiques


In [253]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,sometimes to the devil or the virgin,44.193000,46.193000,sometimes to the devil or the virgin,4nmWfnmVgo4
1,and beauty of its rocks,51.888615,53.288000,the forest of huelgoat is best known for the c...,4nmWfnmVgo4
2,son cours long de moins de,12.322500,15.090000,parsemant son cours long de moins de,BYd2A4JSdOo
3,est un éloge des eaux,15.090000,16.518706,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
4,sont le refuge despèces menacées ou en régress...,22.542289,26.550000,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo
...,...,...,...,...,...
306508,vous pourrez choisir des pistes dactions techn...,223.080000,225.580000,vous pourrez choisir des pistes dactions techn...,oqXlP80c1uk
306509,telles que les objectifs et les horaires au tr...,229.180000,231.760000,telles que les objectifs et les horaires au tr...,oqXlP80c1uk
306510,mais également humaines avec des formations,231.920000,234.560000,mais également humaines avec des formations,oqXlP80c1uk
306511,ou bien encore des échanges de pratiques,234.600000,236.240000,ou bien encore des échanges de pratiques,oqXlP80c1uk


In [254]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,sometimes to the devil or the virgin,43.893000,46.493000,sometimes to the devil or the virgin,4nmWfnmVgo4
1,and beauty of its rocks,51.588615,53.588000,the forest of huelgoat is best known for the c...,4nmWfnmVgo4
2,son cours long de moins de,12.022500,15.390000,parsemant son cours long de moins de,BYd2A4JSdOo
3,est un éloge des eaux,14.790000,16.818706,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
4,sont le refuge despèces menacées ou en régress...,22.242289,26.850000,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo
...,...,...,...,...,...
306508,vous pourrez choisir des pistes dactions techn...,222.780000,225.880000,vous pourrez choisir des pistes dactions techn...,oqXlP80c1uk
306509,telles que les objectifs et les horaires au tr...,228.880000,232.060000,telles que les objectifs et les horaires au tr...,oqXlP80c1uk
306510,mais également humaines avec des formations,231.620000,234.860000,mais également humaines avec des formations,oqXlP80c1uk
306511,ou bien encore des échanges de pratiques,234.300000,236.540000,ou bien encore des échanges de pratiques,oqXlP80c1uk


In [255]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,sometimes to the devil or the virgin,44,46,sometimes to the devil or the virgin,4nmWfnmVgo4
1,and beauty of its rocks,52,54,the forest of huelgoat is best known for the c...,4nmWfnmVgo4
2,son cours long de moins de,12,15,parsemant son cours long de moins de,BYd2A4JSdOo
3,est un éloge des eaux,15,17,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
4,sont le refuge despèces menacées ou en régress...,22,27,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo
...,...,...,...,...,...
306508,vous pourrez choisir des pistes dactions techn...,223,226,vous pourrez choisir des pistes dactions techn...,oqXlP80c1uk
306509,telles que les objectifs et les horaires au tr...,229,232,telles que les objectifs et les horaires au tr...,oqXlP80c1uk
306510,mais également humaines avec des formations,232,235,mais également humaines avec des formations,oqXlP80c1uk
306511,ou bien encore des échanges de pratiques,234,237,ou bien encore des échanges de pratiques,oqXlP80c1uk


In [256]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ici vous devrez apprendre les leçons de chez n...,5564,5630,daccord merci beaucoup john laughland owen pat...,tlPvaDllxiI
1,changer ma vie et de réaliser tout mes rêve et...,0,2,moi cest roland et mon ami ousouf jai contacte...,xdK4jgVZ77c
2,si vous voulez laccord de vendredi saint avec ...,4267,4335,si vous voulez laccord de vendredi saint avec ...,tlPvaDllxiI
3,avec nos voisins et cest fondamental ça parce ...,10156,10222,et ce que nous allons faire nous allons aller ...,tlPvaDllxiI
4,je voudrais juste dire quelque chose ici cest ...,3796,3812,je voudrais juste dire quelque chose ici cest ...,tlPvaDllxiI
...,...,...,...,...,...
306508,on a on a des,1768,1770,on a on a des convergences,seWzxrNRuZc
306509,que a n no ha,204,205,ganancias resultar n en el investimiento del f...,hFw8S3mSYiw
306510,que ça oh o o,81,104,que ça oh o o,ucU8cuYqy24
306511,tu n es qu un,116,118,tu n es qu un numéroet ils auront ta peau,RwctCkPAjOY


In [257]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,ici vous devrez apprendre les leçons de chez n...,5564,5630,daccord merci beaucoup john laughland owen pat...,tlPvaDllxiI,60
1,changer ma vie et de réaliser tout mes rêve et...,0,2,moi cest roland et mon ami ousouf jai contacte...,xdK4jgVZ77c,71
2,si vous voulez laccord de vendredi saint avec ...,4267,4335,si vous voulez laccord de vendredi saint avec ...,tlPvaDllxiI,67
3,avec nos voisins et cest fondamental ça parce ...,10156,10222,et ce que nous allons faire nous allons aller ...,tlPvaDllxiI,79
4,je voudrais juste dire quelque chose ici cest ...,3796,3812,je voudrais juste dire quelque chose ici cest ...,tlPvaDllxiI,75
...,...,...,...,...,...,...
306508,on a on a des,1768,1770,on a on a des convergences,seWzxrNRuZc,60
306509,que a n no ha,204,205,ganancias resultar n en el investimiento del f...,hFw8S3mSYiw,100
306510,que ça oh o o,81,104,que ça oh o o,ucU8cuYqy24,80
306511,tu n es qu un,116,118,tu n es qu un numéroet ils auront ta peau,RwctCkPAjOY,100


In [258]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-258-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-258-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,détruire cette tyrannie cette prison des peupl...,7753,7777,donc il y a une forte résilience nous contribu...,tlPvaDllxiI
1,alors quelle avait tous les atouts dans son je...,4471,4501,et cette andouille de mme may sest présentée l...,tlPvaDllxiI
2,lobjectif est de les amener avec ces outils au...,169,181,lobjectif est de les amener avec ces outils au...,COVrRKxAyRs
3,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo
4,va dans les médias en roulant des mécaniques e...,6474,6492,guy verhofstadt va dans les médias en roulant ...,tlPvaDllxiI
...,...,...,...,...,...
231292,il ny a pas de,1126,1127,il ny a pas de conditionnalités aux engagement...,37uH5CT6fJg
231293,il ny en a pas,177,179,il ny en a pas,vvn1pp6E8ow
231294,il ny en a pas,1290,1294,il ny en a pas,jKV4MNNPxoA
231295,que a n no ha,204,205,ganancias resultar n en el investimiento del f...,hFw8S3mSYiw


In [259]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-259-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,détruire cette tyrannie cette prison des peupl...,7753,7777,donc il y a une forte résilience nous contribu...,tlPvaDllxiI,46
1,alors quelle avait tous les atouts dans son je...,4471,4501,et cette andouille de mme may sest présentée l...,tlPvaDllxiI,62
2,lobjectif est de les amener avec ces outils au...,169,181,lobjectif est de les amener avec ces outils au...,COVrRKxAyRs,83
3,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo,100
4,va dans les médias en roulant des mécaniques e...,6474,6492,guy verhofstadt va dans les médias en roulant ...,tlPvaDllxiI,37
...,...,...,...,...,...,...
231292,il ny a pas de,1126,1127,il ny a pas de conditionnalités aux engagement...,37uH5CT6fJg,25
231293,il ny en a pas,177,179,il ny en a pas,vvn1pp6E8ow,100
231294,il ny en a pas,1290,1294,il ny en a pas,jKV4MNNPxoA,100
231295,que a n no ha,204,205,ganancias resultar n en el investimiento del f...,hFw8S3mSYiw,14


In [260]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-260-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo
1,même si le meilleur moyen de réduire nos déche...,345,354,même si le meilleur moyen de réduire nos déche...,eXE0dzAXErg
2,chassés pour leur fourrure et leurs os utilisé...,529,539,chassés pour leur fourrure et leurs os utilisé...,ykcYguOUSSM
3,bonjour et bienvenue dans cette nouvelle vidéo...,30,38,bonjour et bienvenue dans cette nouvelle vidéo...,u0YcBe6UKGQ
4,donc là tous les gens se réunissent autour de ...,1574,1584,donc là tous les gens se réunissent autour de ...,GUQKIROlp2w
...,...,...,...,...,...
125611,et là on a fait,346,356,et là on a fait,_nHCXVDSet8
125612,il ny en a pas,755,756,il ny en a pas,AervOIKnPmU
125613,ok le mec il a,19,24,ok le mec il a,BlZq5PTo7sg
125614,il ny en a pas,177,179,il ny en a pas,vvn1pp6E8ow


In [261]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,lassemblée nationale monsieur lancien président,zzV2t7s8LnU
1,quils avaient été jusque là,zzV2t7s8LnU
2,pensezvous quil sagisse dun recueil déloges,zzV2t7s8LnU
3,leurope était faite unie contre langleterre,zzV2t7s8LnU
4,napoléon comprit aussi quil devait rechercher ...,zzV2t7s8LnU
...,...,...
2116,aux produits ministres ont la tension était gr...,08ikcqNz89w
2117,bien immobilier quel est mon métier initial qu...,04bzEfxA1i0
2118,jai toujours donné priorité au contenu plutôt ...,04bzEfxA1i0
2119,quel était mon métier ma formation,04bzEfxA1i0


In [262]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo
1,même si le meilleur moyen de réduire nos déche...,345,354,même si le meilleur moyen de réduire nos déche...,eXE0dzAXErg
2,chassés pour leur fourrure et leurs os utilisé...,529,539,chassés pour leur fourrure et leurs os utilisé...,ykcYguOUSSM
3,bonjour et bienvenue dans cette nouvelle vidéo...,30,38,bonjour et bienvenue dans cette nouvelle vidéo...,u0YcBe6UKGQ
4,donc là tous les gens se réunissent autour de ...,1574,1584,donc là tous les gens se réunissent autour de ...,GUQKIROlp2w
...,...,...,...,...,...
2087,ill see what it is,1676,1678,ill see what it is,v3hX1RNA6jA
2088,what is it too hot,85,87,what is it too hot,3_1Scvw8MnA
2089,this is two in one,684,685,this is two in one,3_1Scvw8MnA
2090,we met one of them,81,82,we met one of them,ZfTnwxYSmp4


In [263]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo,https://www.youtube.com/watch?v=e2ITAibdtWo&t=...
1,même si le meilleur moyen de réduire nos déche...,345,354,même si le meilleur moyen de réduire nos déche...,eXE0dzAXErg,https://www.youtube.com/watch?v=eXE0dzAXErg&t=...
2,chassés pour leur fourrure et leurs os utilisé...,529,539,chassés pour leur fourrure et leurs os utilisé...,ykcYguOUSSM,https://www.youtube.com/watch?v=ykcYguOUSSM&t=...
3,bonjour et bienvenue dans cette nouvelle vidéo...,30,38,bonjour et bienvenue dans cette nouvelle vidéo...,u0YcBe6UKGQ,https://www.youtube.com/watch?v=u0YcBe6UKGQ&t=30s
4,donc là tous les gens se réunissent autour de ...,1574,1584,donc là tous les gens se réunissent autour de ...,GUQKIROlp2w,https://www.youtube.com/watch?v=GUQKIROlp2w&t=...
...,...,...,...,...,...,...
2087,ill see what it is,1676,1678,ill see what it is,v3hX1RNA6jA,https://www.youtube.com/watch?v=v3hX1RNA6jA&t=...
2088,what is it too hot,85,87,what is it too hot,3_1Scvw8MnA,https://www.youtube.com/watch?v=3_1Scvw8MnA&t=85s
2089,this is two in one,684,685,this is two in one,3_1Scvw8MnA,https://www.youtube.com/watch?v=3_1Scvw8MnA&t=...
2090,we met one of them,81,82,we met one of them,ZfTnwxYSmp4,https://www.youtube.com/watch?v=ZfTnwxYSmp4&t=81s


In [264]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [265]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [266]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [267]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['French_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'French_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'French_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'French_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'French_Word_Group_Max_In_Youtube_Sentence_10000_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'French_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'French_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [268]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [269]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass